In [9]:
import hddm
import tensorflow as tf
import torch
# Make simulators visible
import sys
import numpy as np
#sys.path.append('simulators')

# Data management
import pandas as pd
import numpy as np
import pickle

# Plotting
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# Stats functionality
from statsmodels.distributions.empirical_distribution import ECDF

In [13]:
model = 'weibull'
data = hddm.simulators.hddm_dataset_generators.simulator_single_subject(parameters = [0.3, 1.5, 0.5, 0.3, 2.0, 0.5], # 0.0, 0, 1.0],
                                                                        model = model)

Model:  weibull
Parameters needed:  ['v', 'a', 'z', 't', 'alpha', 'beta']
[0.3, 1.5, 0.5, 0.3, 2.0, 0.5]
theta is supplied as list --> simulator assumes n_trials = 1
{'v': array([0.3], dtype=float32), 'a': array([1.5], dtype=float32), 'z': array([0.5], dtype=float32), 't': array([0.3], dtype=float32), 's': 1.0, 'alpha': array([2.], dtype=float32), 'beta': array([0.5], dtype=float32), 'delta_t': 0.0010000000474974513, 'max_t': 20.0, 'n_samples': 1000, 'simulator': 'ddm_flexbound', 'boundary_fun_type': 'weibull_cdf', 'possible_choices': [-1, 1], 'trajectory': array([[ 0.000000e+00],
       [-2.291682e-02],
       [-5.616391e-02],
       ...,
       [-9.990000e+02],
       [-9.990000e+02],
       [-9.990000e+02]], dtype=float32), 'boundary': array([1.5      , 1.499994 , 1.4999759, ..., 0.       , 0.       ,
       0.       ], dtype=float32), 'model': 'weibull'}
v
0.3
a
1.5
z
0.5
t
0.3
alpha
2.0
beta
0.5


In [14]:
data[0]

,rt,response,subj_idx,v,a,z,t,alpha,beta
0,0.836997,1,none,0.3,1.5,0.5,0.3,2.0,0.5
1,0.764998,1,none,0.3,1.5,0.5,0.3,2.0,0.5
2,0.583000,1,none,0.3,1.5,0.5,0.3,2.0,0.5
3,0.647999,1,none,0.3,1.5,0.5,0.3,2.0,0.5
4,0.878996,0,none,0.3,1.5,0.5,0.3,2.0,0.5
...,...,...,...,...,...,...,...,...,...
995,0.966995,1,none,0.3,1.5,0.5,0.3,2.0,0.5
996,0.723998,0,none,0.3,1.5,0.5,0.3,2.0,0.5
997,0.830997,1,none,0.3,1.5,0.5,0.3,2.0,0.5
998,0.687999,0,none,0.3,1.5,0.5,0.3,2.0,0.5


In [15]:
my_dat = data[0]
my_dat['covariate'] = np.random.uniform(size = my_dat.shape[0])

In [16]:
reg_models = [{'model': 'v ~ 1 + covariate', 'link_func': lambda x: x}]

In [17]:
model_ = hddm.HDDMnnRegressor(my_dat, 
                             reg_models,
                             include = hddm.model_config.model_config[model]['hddm_include'],
                             model = model,
                             informative = False,
                             p_outlier = 0.1)

Setting priors uninformative (LANs only work with uninformative priors for now)
tanh
tanh
tanh
linear
def custom_likelihood_reg(value, v, a, z, t, alpha, beta, reg_outcomes, p_outlier=0, w_outlier=0.1, **kwargs):
    params = locals()
    size = int(value.shape[0])
    data = np.zeros(((size, 8)), dtype=np.float32)
    data[:, 6:] = np.stack([np.absolute(value["rt"]).astype(np.float32), value["response"].astype(np.float32)], axis=1)
    cnt=0
    for tmp_str in ['v', 'a', 'z', 't', 'alpha', 'beta']:
        if tmp_str in reg_outcomes:
            data[:, cnt] = params[tmp_str].loc[value["rt"].index].values
            if (data[:, cnt].min() < [-2.5, 0.3, 0.2, 0.001, 0.31, 0.31][cnt]) or (data[:, cnt].max() > [2.5, 2.5, 0.8, 2.0, 4.99, 6.99][cnt]):
                print("boundary violation of regressor part")
                return -np.inf
        else:
            data[:, cnt] = params[tmp_str]
        cnt += 1
    return hddm.wfpt.wiener_like_multi_nn_mlp(data, p_outlier=p_outlier, w_

In [18]:
model_.sample(1000, burn = 500)

 [-----------------100%-----------------] 1000 of 1000 complete in 52.7 sec

In [23]:
model_.gen_stats()

,mean,std,2.5q,25q,50q,75q,97.5q,mc err
a,1.84925,0.118533,1.589446,1.773494,1.870586,1.936405,2.049134,0.011312
z,0.488417,0.014971,0.459062,0.478482,0.487824,0.498712,0.518703,0.001309
t,0.189547,0.034984,0.130627,0.160479,0.189606,0.210837,0.265371,0.003466
alpha,2.444425,0.165564,2.130347,2.327868,2.44461,2.551969,2.767661,0.015283
beta,0.581609,0.02895,0.523176,0.562331,0.580845,0.601984,0.635017,0.002854
v_Intercept,0.345363,0.118282,0.0848,0.267929,0.352107,0.431306,0.558493,0.010649
v_covariate,-0.009511,0.131921,-0.264974,-0.099813,-0.015535,0.083716,0.237354,0.010777


In [20]:
model__ = hddm.HDDMRegressor(my_dat, 
                     reg_models,
                     include = ['z'], #hddm.model_config.model_config[model]['hddm_include'],
                     informative = False,
                     p_outlier = 0.1)

Adding these covariates:
['v_Intercept', 'v_covariate']
{'group_only_nodes': ['v_Intercept', 'v_covariate']}
['z']
No model attribute --> setting up standard HDDM
Includes supplied:  ['z']
printing self.nn
False
Set model to ddm
passing through _create_stochastic_knodes
Reg Model:
{'outcome': 'v', 'model': ' 1 + covariate', 'params': ['v_Intercept', 'v_covariate'], 'link_func': <function <lambda> at 0x7f41c821d050>}
Uses Identity Link


In [22]:
model__.gen_stats()

,mean,std,2.5q,25q,50q,75q,97.5q,mc err
a,1.058152,0.015673,1.029114,1.047127,1.058348,1.068934,1.088845,0.000854
t,0.522855,0.002111,0.518315,0.521595,0.522908,0.524278,0.526777,0.00012
z,0.49693,0.011971,0.474914,0.488451,0.496917,0.505591,0.519281,0.000743
v_Intercept,0.390914,0.139088,0.112379,0.294693,0.394179,0.490947,0.662364,0.011404
v_covariate,-0.07869,0.240448,-0.570889,-0.240938,-0.061337,0.075219,0.421024,0.019329


In [21]:
model__.sample(1000, burn = 500)

 [-----------------100%-----------------] 1000 of 1000 complete in 36.6 sec

In [4]:
import pickle
x = pickle.load(open('/users/afengler/data/proj_lan_pipeline/LAN_scripts/config_files/ddm_mic2_adj_angle_no_bias_nsim_200000_dt_0.001_nps_1000_npts_2000.pickle', 'rb'))

In [5]:
x

{'model_config': {'name': 'ddm_mic2_angle_no_bias',
  'params': ['vh', 'vl1', 'vl2', 'a', 'd', 't', 'theta'],
  'param_bounds': [[-2.5, -2.5, -2.5, 0.3, 0.0, 0.0, -0.1],
   [2.5, 2.5, 2.5, 2.0, 1.0, 2.0, 1.0]],
  'boundary': <function ssms.basic_simulators.boundary_functions.angle(t=1, theta=1)>,
  'boundary_multiplicative': False,
  'n_params': 7,
  'default_params': [0.0, 0.0, 0.0, 1.0, 0.5, 1.0, 0.0],
  'hddm_include': ['vh',
   'vl1',
   'vl2',
   'a',
   'zh',
   'zl1',
   'zl2',
   'd',
   't',
   'theta'],
  'nchoices': 4},
 'data_config': {'output_folder': 'data/lan_mlp/',
  'dgp_list': 'ddm_mic2_adj_angle_no_bias',
  'nbins': 0,
  'n_samples': 200000,
  'n_parameter_sets': 1000,
  'n_parameter_sets_rejected': 100,
  'n_training_samples_by_parameter_set': 2000,
  'max_t': 20.0,
  'delta_t': 0.001,
  'pickleprotocol': 4,
  'n_cpus': 'all',
  'kde_data_mixture_probabilities': [0.8, 0.1, 0.1],
  'simulation_filters': {'mode': 20,
   'choice_cnt': 5,
   'mean_rt': 15,
   'std': 0,
